In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Iteration4').getOrCreate()

In [3]:
# Reading the datasets
dataset_one = spark.read.csv('./student-math-dataset1.csv', header=True, inferSchema=True)
dataset_two = spark.read.csv('./student-math-dataset2.csv', header=True, inferSchema=True)

In [10]:
# 2.2 Describe the data
print("Dataset one structure: " + str(dataset_one.count()) + " rows, and " + str(len(dataset_one.columns)) + " columns")
print("Dataset two structure: " + str(dataset_two.count()) + " rows, and " + str(len(dataset_two.columns)) + " columns")
print("How many students inside the dataset: " + str(dataset_one.count()))

Dataset one structure: 395 rows, and 31 columns
Dataset two structure: 395 rows, and 4 columns
How many students inside the dataset: 395


In [11]:
print("Data type inside the dataset one")
dataset_one.dtypes

Data type inside the dataset one


[('id', 'int'),
 ('school', 'string'),
 ('sex', 'string'),
 ('age', 'int'),
 ('address', 'string'),
 ('famsize', 'string'),
 ('Pstatus', 'string'),
 ('Medu', 'int'),
 ('Fedu', 'int'),
 ('Mjob', 'string'),
 ('Fjob', 'string'),
 ('reason', 'string'),
 ('guardian', 'string'),
 ('traveltime', 'int'),
 ('studytime', 'int'),
 ('failures', 'int'),
 ('schoolsup', 'string'),
 ('famsup', 'string'),
 ('paid', 'string'),
 ('activities', 'string'),
 ('nursery', 'string'),
 ('higher', 'string'),
 ('internet', 'string'),
 ('romantic', 'string'),
 ('famrel', 'int'),
 ('freetime', 'double'),
 ('goout', 'int'),
 ('Dalc', 'double'),
 ('Walc', 'double'),
 ('health', 'double'),
 ('absences', 'int')]

In [12]:
print("Data type inside the dataset two")
dataset_two.dtypes

Data type inside the dataset two


[('id', 'int'), ('G1', 'int'), ('G2', 'int'), ('G3', 'int')]